<a href="https://colab.research.google.com/github/junior825/BERT/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"parkyongchan","key":"3a4d692af68deb0ef0e7c9f388541a58"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         87           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        107           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        892           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        160           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1847           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      13876           False  
house-pr

In [ ]:
!kaggle competitions download -c nlp-getting-started

!kaggle competitions files -c nlp-getting-started

test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
name                    size  creationDate         
---------------------  -----  -------------------  
train.csv              965KB  2019-12-16 20:36:20  
test.csv               411KB  2019-12-16 20:36:20  
sample_submission.csv   22KB  2019-12-16 20:36:20  


In [ ]:
import pandas as pd

test = pd.DataFrame(pd.read_csv('./test.csv')) # test feature
submission = pd.DataFrame(pd.read_csv('./sample_submission.csv')) # Prediction
train = pd.DataFrame(pd.read_csv('./train.csv')) # train_tata => feature / target

train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
print(train.isnull().sum())
print(len(train))
print(train.columns)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
7613
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [ ]:
train_data = train.drop(['id','keyword','location'], axis=1)

train_data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [ ]:
print(train_data.groupby('target').size())

target
0    4342
1    3271
dtype: int64


In [ ]:
import re

print(train_data['text'])
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!·]', ' ', inputString) 
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

train_text_feature = [clean_text(str(s)) for s in train_data['text']]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

train_feature, val_feature, train_target, val_target = train_test_split(train_text_feature, train_data['target'], train_size = 0.8, random_state = 40)

print(train_feature[0])

flood disaster Heavy rain causes flash flooding of streets in Manitou Colorado Springs areas


In [ ]:
!pip install transformers==4.6.0
from transformers import BertTokenizerFast, TFBertModel

BERT_MODEL = 'bert-base-uncased'

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL)

train_tokenizer = tokenizer(train_feature, return_tensors='tf', truncation = True, padding = True)
val_tokenizer = tokenizer(val_feature, return_tensors='tf', truncation = True, padding = True)

In [ ]:
print(train_tokenizer[0].tokens)

['[CLS]', 'flood', 'disaster', 'heavy', 'rain', 'causes', 'flash', 'flooding', 'of', 'streets', 'in', 'mani', '##tou', 'colorado', 'springs', 'areas', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
# from_tensor_slice는 train_target 1개 안에 train_tokenizer(input_ids, token_type_ids, attention_mask) 3개의 Dictionary로 묶도록 만들어주는 역할을 수행함.
# NLP 분야 뿐만이 아니라 다른 분야에서도 이렇게 Dic으로 묶어놓는게 훨씬 보기 좋고 편하다고 판단됨

import tensorflow as tf

with tf.device('/device:GPU:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokenizer),
    train_target
    ))
  
  val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_tokenizer),
    val_target
    ))

In [ ]:
print(train_dataset)

<TensorSliceDataset shapes: ({input_ids: (59,), token_type_ids: (59,), attention_mask: (59,)}, ()), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import BertConfig # BERT의 기본 Model에 layer / Node / drop_out 등을 조정할 수 있게 하는 Model
from transformers import BertModel

def get_model():
  config = BertConfig.from_pretrained(BERT_MODEL, 
                                      hidden_dropout_prob = 0.2,
                                      # Attention_probs_dropout_prob = 0.4,
                                      num_labels=2)
  model = TFBertForSequenceClassification.from_pretrained(BERT_MODEL, config=config, from_pt=True)
  return model
  
model = get_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer = optimizer, loss = model.compute_loss, metrics=['accuracy']) # loss를 변경하니까 accuracy가 확실하게 올라간다...? 이게 무슨일인가? model.comput_loss는 tensorflow 전용으로 사용되는 loss로써 test와 train을 step마다 변화시켜준다.(이론은 들여다봐야할듯)

with tf.device('/device:GPU:0'): # device에 있는 GPU:0를 사용하겠다.
  history = model.fit(
    train_dataset.batch(16), 
    validation_data = val_dataset.batch(16),
    epochs = 5
    )

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
# Transformer 자체에 문제가 있어 이전 버젼인 4.6.0으로 변환 후에 TFTrainer를 돌려야 가능하다.
# Trainer의 장점 => model.fit은 learning_rate가 똑같은 속도로 학습되나 Trainer의 경우에는 warm_up_step에 맞게 learning_rate가 지속적으로 변화하면서 학습됨.
# Trainer의 경우 Upstream이 있고 Downstream이 있는데  0 => 지정한 속도까지 학습이냐 지정한 속도 => 0 으로 학습시키냐의 차이로 보여짐. 추가적인 공부가 필요함.

from transformers import TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments( # 해당 function에 대한 구체적인 공부 필요
    output_dir='./results',          # output directory
    num_train_epochs= 5,              # total number of training epochs
    per_device_train_batch_size= 16,  # batch size per device during training
    per_device_eval_batch_size= 64,   # batch size for evaluation
    warmup_steps= 500,                # number of warmup steps for learning rate scheduler
    weight_decay= 0.01,               # strength of weight decay
    logging_dir='./logs'            # directory for storing logs
)

with training_args.strategy.scope():
    trainer_model = TFBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2, from_pt=True)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

# Trainer의 경우는 Accuaracy / loss가 나타나지 기울기/혼합정밀도 등을 사용할 수 있음.(W&B DashBoard를 추천한다.) => 이후 알아야 하는 부분

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
print(train_dataset)
print(train_dataset.batch(16))

In [ ]:
# hidden_dropout_prob

In [ ]:
print(history.history)

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.legend(['Train','Test'], loc = 'upper left')
plt.show()

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Load')
plt.legend()
plt.legend(['Train','Test'], loc = 'upper left')
plt.show()

In [ ]:
import os
MODEL_NAME = 'My-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

In [ ]:
test['real'] = submission['target']

test

In [ ]:
predicted_label_list = []
predicted_score_list = []

for text in test['text']:
    # predict
    preds_list = text_classifier(text)[0]

    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    predicted_label_list.append(sorted_preds_list[0]) # label
    predicted_score_list.append(sorted_preds_list[1]) # score

test['pred'] = predicted_label_list
test['score'] = predicted_score_list
test.head()

#result_data = pd.DataFrame({'text' : test['text'], 'target' : submission['target'], 'predict' : predict})